In [34]:
%pip install -q pypdf sentence-transformers langchain-openai faiss-gpu rank_bm25

ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain.document_loaders.pdf import PyPDFLoader

In [3]:
loader = PyPDFLoader("./assets/비자 내비게이터(Visa Navigator) 전자책(국문).pdf")

In [4]:
docs = loader.load_and_split()

print(len(docs))
# page = pages[4]
# print(page.page_content)

58


In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500, 
    chunk_overlap  = 50,
)
texts = text_splitter.split_documents(docs)

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli"
encode_kwargs = {'normalize_embeddings': True}
ko_embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

/Users/heewungsong/anaconda3/envs/pyautogen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
bm25_retriever = BM25Retriever.from_documents(texts)
bm25_retriever.k = 2

In [8]:
from langchain_community.vectorstores import FAISS

embedding = ko_embedding
faiss_vectorstore = FAISS.from_documents(texts, ko_embedding)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 2})

# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

In [41]:
docs = ensemble_retriever.invoke("D-2비자와 D-4비자의 차이")
for i in docs:

  print(i.metadata)
  print(":")
  print(i.page_content)
  print("-"*100)

{'source': './assets/비자 내비게이터(Visa Navigator) 전자책(국문).pdf', 'page': 36}
:
37
PART 02. 국내 체류 외국인을 위한 맞춤형 체류 길잡이 ●특정 체류자격에서 F-2(거주) 자격으로 변경한 경우
- 취재(D-5), 주재(D-7), 기업투자(D-8), 무역경영(D-9), 교수(E-1)~특정활동(E-7) 자격자가 해당 자격의 
활동범위를 유지하며 거주 체류자격(F-2)으로 변경한 경우에 한해 연간소득이 전년도 GNI 2배 이상인 
경우 부모(배우자의 부모 포함)를 F-1 자격으로 초청 가능 (단, 국내 대학에서 학사학위 이상을 취득한 후, 
상기 체류자격으로 변경한 자는 GNI 이상으로 완화)
※ 해당 자격의 경우, 원칙적으로 국내 체류자격 변경 불가하며 사증발급인정서 신청 대상
 ●그 외 F-2 비자
- 배우자 및 미성년 자녀를 방문동거(F-1) 자격으로 초청 가능
----------------------------------------------------------------------------------------------------
{'source': './assets/비자 내비게이터(Visa Navigator) 전자책(국문).pdf', 'page': 57}
:
58
비자 내비게이터 ●근무처변경·추가 허가, 신고
핵심내용외국인이 체류자격의 범위에서 근무처를 변경하거나 추가하려면 미리 허가를 받아야 함 
(단, 신고 대상인 경우 근무처를 변경하거나 추가한 날부터 15일 이내 신고)
위반사례회화지도 강사가 계약기간 끝나지 않은 채로 근무하는 학원을 변경하였으나 신고하지 
않은 경우
결과▪ 위반법조  출입국관리법 제21조제1항 위반
▪ 처분내용  	허가 대상의 경우 범칙금 최대 1000만원, 신고 대상의 경우 과태료 최대 
200만원
 ●외국 국적 취득 후 한국 여권 사용 제한
핵심내용 외국 국적을 취득한 경우 한국 여권을 사용하지 않아야 함
위반사례 미국 시민권을 취득한 사람이 한국 입국 심사 

In [42]:
faiss_vectorstore = FAISS.from_documents(texts, ko_embedding)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 4})

docs = faiss_retriever.invoke("D-2비자와 D-4비자의 차이")
for i in docs:

  print(i.metadata)
  print(":")
  print(i.page_content)
  print("-"*100)

{'source': './assets/비자 내비게이터(Visa Navigator) 전자책(국문).pdf', 'page': 36}
:
37
PART 02. 국내 체류 외국인을 위한 맞춤형 체류 길잡이 ●특정 체류자격에서 F-2(거주) 자격으로 변경한 경우
- 취재(D-5), 주재(D-7), 기업투자(D-8), 무역경영(D-9), 교수(E-1)~특정활동(E-7) 자격자가 해당 자격의 
활동범위를 유지하며 거주 체류자격(F-2)으로 변경한 경우에 한해 연간소득이 전년도 GNI 2배 이상인 
경우 부모(배우자의 부모 포함)를 F-1 자격으로 초청 가능 (단, 국내 대학에서 학사학위 이상을 취득한 후, 
상기 체류자격으로 변경한 자는 GNI 이상으로 완화)
※ 해당 자격의 경우, 원칙적으로 국내 체류자격 변경 불가하며 사증발급인정서 신청 대상
 ●그 외 F-2 비자
- 배우자 및 미성년 자녀를 방문동거(F-1) 자격으로 초청 가능
----------------------------------------------------------------------------------------------------
{'source': './assets/비자 내비게이터(Visa Navigator) 전자책(국문).pdf', 'page': 13}
:
유형1 주재, 투자, 경영 자격에서 영주 자격 취득
주재·투자·경영
D-7, D-8, D-9거주
F-2영주
F-5
----------------------------------------------------------------------------------------------------
{'source': './assets/비자 내비게이터(Visa Navigator) 전자책(국문).pdf', 'page': 31}
:
32
비자 내비게이터8. 거주(F-2)
❶ 거주 비자의 종류
체류자격 세부 대상
F-2
(거주)• 국민의 미성년 자녀, 국민과 혼인관계에서 출생한 사람
• 영주자격 소지자의 배우자 및 미성년 자녀
• 난민 인정을 

In [9]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI


def gpt_call(model_name: str, query: str):
    llm = ChatOpenAI(model_name=model_name, temperature = 0)
    qa = RetrievalQA.from_chain_type(
        llm = llm, 
        chain_type = "stuff", 
        retriever = ensemble_retriever,
        return_source_documents = True
    )
    # "난 베트남에서 한국으로 공부하러 온 유학생인데 아직은 학생이지만 여기에서 영주권을 얻어서 살고 싶어. 그러기 위해선 어떤 준비가 필요해?"
    result = qa(query)
    return result

In [56]:
result = gpt_call(model_name="gpt-3.5-turbo")

print(f"Result: {result['result']}\n")
for i in result['source_documents']:
    print(i.metadata)
    print("-"*100)
    print(i.page_content)
    print("-"*100)
    print("\n\n")

Result: 유학생이 한국에서 영주권을 획득하려면 일정한 절차와 요건을 충족해야 합니다. 예를 들어, 사회통합프로그램을 이수하거나 토픽 4급 이상의 한국어 능력을 증명하는 등의 요건이 있을 수 있습니다. 따라서, 영주권 획득을 위한 자세한 정보는 사회통합정보망(socinet) 홈페이지나 외국인종합안내센터(☎1345)를 통해 확인하시는 것이 좋습니다.

{'source': './assets/비자 내비게이터(Visa Navigator) 전자책(국문).pdf', 'page': 49}
----------------------------------------------------------------------------------------------------
50
비자 내비게이터해외 출국 중인데 체류기간연장, 체류자격변경 등이 가능한가요?
 각종 체류허가 신청을 하고자 하는 외국인은 반드시 국내에 체류 중에 있어야 하며, 출국 
중에는 출입국민원 대행기관의 대행 신청도 할 수 없습니다.
사회통합프로그램이란 무엇이며, 이수할 경우 어떤 혜택이 있나요?
 사회통합프로그램이란 대한민국에 체류하는 이민자가 우리 사회 구성원으로 적응, 
자립하는데 필요한 기본소양을 함양할 수 있도록 마련한 교육이며, 이수할 경우 영주 자격 
신청 시 기본소양 요건 충족, 그 외 체류자격 신청 시 점수제의 경우 가점 부여 등 혜택이 
있습니다. 자세한 내용은 사회통합정보망(socinet) 홈페이지를 참고하시기 바랍니다.
조기적응 프로그램이란 무엇이며, 이수할 경우 어떤 혜택이 있나요?
 조기적응 프로그램은 입국 초기 단계의 외국인에게 한국 생활에 필수적인 기초법, 질서를
----------------------------------------------------------------------------------------------------



{'source': './assets/비자 내비게이터(Visa Navigator) 전자책(국문).pdf', 'page':

In [57]:
result2= gpt_call(model_name="gpt-4")
print(f"Result2: {result2['result']}\n")

Result2: 한국에서 영주권을 얻기 위해서는 여러 가지 요건을 충족해야 합니다. 먼저, 한국에서 학사 학위 이상을 취득해야 합니다. 그리고 사회통합프로그램을 이수하면 영주 자격 신청 시 기본소양 요건 충족과 체류자격 신청 시 점수제의 경우 가점 부여 등의 혜택을 받을 수 있습니다. 

또한, 국내 정규 대학에서 전문학사 이상 학위를 취득한 후 3년이 경과되지 않은 경우, 토픽 4급 이상 유효 성적표를 소지하거나 사회통합 프로그램 4단계 중간평가 합격자 또는 사전평가 5단계 배정자는 점수제 면제를 받을 수 있습니다.

이 외에도, 한국에서 취업을 하거나, 기술창업을 준비하는 등 다양한 방법으로 체류자격을 변경하거나 연장하는 것이 가능합니다. 하지만 이 모든 과정은 한국에 체류 중일 때 가능하며, 출국 중에는 신청이 불가능합니다.

더 자세한 정보는 사회통합정보망(socinet) 홈페이지나 하이코리아의 체류자격별 안내매뉴얼을 참고하시거나 외국인종합안내센터(☎1345)를 이용하시기 바랍니다.



In [12]:
result= gpt_call(model_name="gpt-4", query="영주권(F-5)을 신청하기 위해 필요한 한국어 능력 시험 점수는 어떻게 되나요?")
print(f"Result: {result['result']}\n")

for i in result['source_documents']:
    print(i.metadata)
    print("-"*100)
    print(i.page_content)
    print("-"*100)
    print("\n\n")

Result: 영주권(F-5)을 신청하기 위한 한국어 능력 시험 점수에 대한 구체적인 정보는 제공된 문맥에서 찾을 수 없습니다. 그러나 일반적으로 사회통합프로그램 5단계 이상을 이수해야 기본소양 요건을 충족할 수 있다고 언급되어 있습니다. 이는 한국어 능력을 포함할 수 있으나, 구체적인 시험 점수나 레벨에 대한 정보는 제공되지 않았습니다.

{'source': './assets/비자 내비게이터(Visa Navigator) 전자책(국문).pdf', 'page': 14}
----------------------------------------------------------------------------------------------------
- (F-5, 점수제 영주자)  점수제 거주(F-2) 자격으로 3년 이상 국내 체류
* 일반영주, 점수제  영주자의 경우 사회통합프로그램 5단계 이상 이수할 경우 기본소양 요건 
충족유형1    주재, 투자, 경영 자격에서 영주 자격 취득
유형2 고액투자자의 영주 자격 취득
투자
D-8거주
F-2영주
F-5
----------------------------------------------------------------------------------------------------



{'source': './assets/비자 내비게이터(Visa Navigator) 전자책(국문).pdf', 'page': 10}
----------------------------------------------------------------------------------------------------
11
PART 02. 국내 체류 외국인을 위한 맞춤형 체류 길잡이유형2 유학생이 졸업  후 구직 자격 취득
유학·연수
D-2, D-4구직
D-10
 1단계 (유학·연수 → 구직)
- (D-10, 일반구직)  ① 교수(E-1)부터 특정활동(E-7) 자격에 해당하는 분야에 취업하기 위해 
연수나 구직활동을 하려는 자로서 학사 학